## **Промежуточная аттестация 4**

Представленне наборы данных содержат:
1. Информацию о 60000 образцах написания рукописных цифр и метки классов для обучения
2. 10000 образцов и меток для тестирования

Каждая строка состоит из 785 значений: первое значение (колонка `label`) — метка (число от 0 до 9), а оставшиеся 784 значения — значения пикселей (числа от 0 до 255).

### 1. Подготовка данных

In [48]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier

Считываем тренировочные данные

In [29]:
train = pd.read_csv('MNIST_train.csv')
train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Считываем тестовые данные

In [30]:
test = pd.read_csv('MNIST_test.csv')
test

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Из тренировочного и тестового наборов данных отберем изображения, относящиеся к классу `7` и `9`

In [31]:
train = train[(train['label'] == 7) | (train['label'] == 9)].reset_index(drop=True)
test = test[(test['label'] == 7) | (test['label'] == 9)].reset_index(drop=True)

print(f'Длина после отбора\ntrain: {len(train)}\ntest: {len(test)}')

Длина после отбора
train: 12214
test: 2037


Отделим значения пикселей от меток классов

In [32]:
X_train, y_train = train.drop(columns='label'), train['label']
X_test, y_test = test.drop(columns='label'), test['label']

print('Форма       X         y')
print(f'train {X_train.shape} {y_train.shape}')
print(f'test {X_test.shape} {y_test.shape}')

Форма       X         y
train (12214, 784) (12214,)
test (2037, 784) (2037,)


Используем метод главных компонент `PCA()` с параметром `svd_solver='full'` на тренировочных данных

In [33]:
pca = PCA(svd_solver='full').fit(X_train)

Определим минимальное количество главных компонент для того, чтобы доля объясненной дисперсии была строго больше `0.84`

In [34]:
n_comp = (pca.explained_variance_ratio_.cumsum() > 0.84).argmax() + 1
print(f'Минимальное количество ГК: {n_comp}')

Минимальное количество ГК: 42


Заново обучим модель `PCA(svd_solver='full')` на тренировочных данных с использованием полученного числа ГК. Применим полученное преобразование для тренировочных и тестовых данных

In [35]:
pca = PCA(n_components=n_comp, svd_solver='full').fit(X_train)

In [36]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

Найдем координату относительно `6` главной компоненты для `10` изображения в тренировочном наборе данных:

In [37]:
print('Координата 10 изображения относительно 6 ГК:', round(X_train[10 - 1][6 - 1], 3))

Координата 10 изображения относительно 6 ГК: -242.799


### 2. Обучение моделей

Обучим модель на основе `sklearn.ensemble.StackingClassifier`. Базовые алгоритмы: `RandomForestClassifier(random_state=9)`, `SVC(random_state=9)`. Решающий: `LogisticRegression(random_state=9)`

In [39]:
estimators = [('Random Forest', RandomForestClassifier(random_state=9)),
              ('SVM', SVC(random_state=9))]
final_estimator = LogisticRegression(random_state=9)

stack = StackingClassifier(estimators, final_estimator).fit(X_train, y_train)

Построим `classification_report` для полученной модели на тренировочных данных

In [44]:
prediction_train = stack.predict(X_train)
print(classification_report(y_train, prediction_train, digits=4))

              precision    recall  f1-score   support

           7     0.9957    0.9958    0.9958      6265
           9     0.9956    0.9955    0.9955      5949

    accuracy                         0.9957     12214
   macro avg     0.9957    0.9957    0.9957     12214
weighted avg     0.9957    0.9957    0.9957     12214



Построим `classification_report` для полученной модели на тестовых данных

In [45]:
prediction_test = stack.predict(X_test)
print(classification_report(y_test, prediction_test, digits=4))

              precision    recall  f1-score   support

           7     0.9961    0.9922    0.9942      1028
           9     0.9921    0.9960    0.9941      1009

    accuracy                         0.9941      2037
   macro avg     0.9941    0.9941    0.9941      2037
weighted avg     0.9941    0.9941    0.9941      2037



Построим матрицу ошибок для полученной модели на тестовых данных

In [47]:
conf_matrix = confusion_matrix(y_test, prediction_test)
print(f'tn, fp, fn, tp: {conf_matrix.ravel()}')

tn, fp, fn, tp: [1020    8    4 1005]


Обучим модель `sklearn.ensemble.AdaBoostClassifier`, используя `DecisionTreeClassifier` на тренировочных данных

In [50]:
tree = DecisionTreeClassifier(max_depth=3, class_weight='balanced')

ada = AdaBoostClassifier(estimator=tree,
                         n_estimators=300,
                         learning_rate=0.5,
                         random_state=9).fit(X_train, y_train)

Построим `classification_report` для полученной модели на тренировочных данных

In [51]:
prediction_train = ada.predict(X_train)
print(classification_report(y_train, prediction_train, digits=4))

              precision    recall  f1-score   support

           7     1.0000    1.0000    1.0000      6265
           9     1.0000    1.0000    1.0000      5949

    accuracy                         1.0000     12214
   macro avg     1.0000    1.0000    1.0000     12214
weighted avg     1.0000    1.0000    1.0000     12214



Построим `classification_report` для полученной модели на тестовых данных

In [52]:
prediction_test = ada.predict(X_test)
print(classification_report(y_test, prediction_test, digits=4))

              precision    recall  f1-score   support

           7     0.9835    0.9835    0.9835      1028
           9     0.9832    0.9832    0.9832      1009

    accuracy                         0.9833      2037
   macro avg     0.9833    0.9833    0.9833      2037
weighted avg     0.9833    0.9833    0.9833      2037



Построим матрицу ошибок для полученной модели на тестовых данных

In [53]:
conf_matrix = confusion_matrix(y_test, prediction_test)
print(f'tn, fp, fn, tp: {conf_matrix.ravel()}')

tn, fp, fn, tp: [1011   17   17  992]
